In [1]:
from requests_html import HTMLSession
import sys
import urllib.request, urllib.error, urllib.parse
import webbrowser
from bs4 import BeautifulSoup
import pyautogui
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.support.ui import WebDriverWait
import re
import enchant
import shutil
import pandas as pd
from statistics import mean 
import os
from urllib.request import Request, urlopen
import urllib.parse
from collections import OrderedDict
import names
import random
import csv
from csv import writer

In [2]:
#Take designer keywords from Poshmark website and paste them into list 
#format: ['Rick Owens','Ann Demeulemeester','Givenchy','Tom Brown']
#reccomend leaving this one running overnight- slow
#can be sped up by commenting out time.sleep on final line of getthebrand function but not cool to do
brands=['Rick Owens']
owd=os.getcwd()
owd

'/home/a/Desktop/poshmarkspider'

In [3]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [4]:

def checkpage(pageurl): 
    thispage=1
    start=True
    while start:
        url=pageurl+str(thispage)
        user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
        values = {'name' : names.get_full_name(),
            'location' : 'Northampton',
            'language' : 'Python' }
        headers = { 'User-Agent' : user_agent }
        data = urllib.parse.urlencode(values)
        data = data.encode('ascii')
        req = Request(url, headers=headers)
        the_page = urlopen(req).read()
        pageitemsoup = BeautifulSoup(the_page, 'lxml')
        finalpage=pageitemsoup.findAll('button',attrs = {'class' : 'btn btn--pagination'})[-2]
        lastpage=int(str(finalpage).split()[-2])+1
        thepage=pageitemsoup.findAll('button',attrs = {'class' : 'btn btn--pagination bg--light-gray fw--med'})
        inpage=int(str(thepage).split()[-2])
        if inpage!=lastpage:
            start = True
            thispage+=1
        else:
            start= False
    return(thispage)

In [5]:
def getthebrand(labels):
    for brand in labels:
        page=1
        directoryname=brand.replace(" ","")
        labelcheck=brand.replace(" ","-")
        os.chdir(owd)
        if not os.path.exists(directoryname):
            os.makedirs(directoryname)
        os.chdir(directoryname)
        if not os.path.exists(directoryname+"catalogue.csv"):
            append_list_as_row(directoryname+"catalogue.csv",["Brand", "ItemCode","URL","Price","Size","Catagories","Colour"])
        branddirect = brand.replace(" ","%20").lower()
        for i in range(1,checkpage("https://poshmark.com/search?query="+branddirect+"&type=listings&max_id=")+1):
            url="https://poshmark.com/search?query="+branddirect+"&type=listings&max_id="+str(i)
            print(i)
            user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
            values = {'name' : names.get_full_name(),
                'location' : 'Northampton',
                'language' : 'Python' }
            headers = { 'User-Agent' : user_agent }
            data = urllib.parse.urlencode(values)
            data = data.encode('ascii')
            req = Request(url, headers=headers)
            the_page = urlopen(req).read()
            pageitemsoup = BeautifulSoup(the_page, 'lxml')
            theurls=[]
            itemurls=[]
            links=pageitemsoup.findAll('a',attrs = {'href' : True,'class' : True})
            for link in links:
                if '/listing/' in str(link['href']):
                    theurls.append(link['href'])
            for itemurl in theurls:
                if (labelcheck in itemurl) or (labelcheck.upper() in itemurl):
                    findurl='https://poshmark.com'+itemurl
                    itemurls.append(findurl)
            for item in itemurls:
                spliturl=item.split("-")
                itemcode=spliturl[-1]
                if not os.path.exists(itemcode):
                    print(item)
                    os.makedirs(itemcode)
                    item_req = Request(item, headers=headers)
                    item_page = urlopen(item_req).read()
                    item_soup = BeautifulSoup(item_page, 'lxml')
                    theprice=item_soup.findAll('h1')
                    thepricetag=str(theprice[-1])
                    thepriceis=(thepricetag.split()[1])
                    catagories=item_soup.findAll('a',attrs = {'class' : "btn btn--tag tag-details__btn",'href' : True})[-1]
                    catagoriesis=(str(catagories['href']).split("/")[-1])
                    colour=item_soup.findAll('a',attrs = {'data-et-prop-listing_color' : True,'href' : True})
                    if len(colour)>0:
                        colour=colour[0]
                        colouris=(str(colour['data-et-prop-listing_color']))
                    else:
                        colouris="N/A"
                    size=item_soup.findAll('button',attrs = {'class' : "size-selector__size-option btn btn--tertiary tc--lb fw--bold br--burgundy"})
                    if len(size)>0:
                        size=size[0]
                        splitsize=str(size).split()
                        sizeis=splitsize[-2]
                    else: 
                        sizeis="N/A"
                    append_list_as_row(directoryname+"catalogue.csv",[brand, itemcode,item,thepriceis,sizeis,catagoriesis,colouris])
                    carouselli=item_soup.findAll('li',attrs = {'class' : "carousel__item carousel__item",'style' : True})
                    for element in carouselli:
                        start=str(element).index("data-src")+10
                        end=str(element).index(" src")-1
                        item_pic_url=(str(element)[start:end])
                        finalimg=item_pic_url.split("/")
                        urllib.request.urlretrieve(str(item_pic_url), str(finalimg[-1]))
                        shutil.move(os.getcwd()+'/'+finalimg[-1], os.getcwd()+'/'+itemcode+'/'+finalimg[-1])
                        time.sleep(random.randint(0,2))

In [6]:
getthebrand(brands)

1
https://poshmark.com/listing/NWT-Rick-Owens-Raglan-Bomber-Sz-50-5eb86b9d7f80d29e15ca605b
https://poshmark.com/listing/Rick-Owens-oversized-tailored-woven-vest-waistcoat-5eb77df44fd23aa79a3cc95e
https://poshmark.com/listing/RICK-OWENS-Velvet-Bias-Pant-wide-leg-5e80ecf2800f6448ce16ff93
https://poshmark.com/listing/Rick-Owens-plum-on-the-bias-velvet-midi-skirt-6-5ec6f5f83a0db925e1939f03
https://poshmark.com/listing/Rick-Owens-6-black-strapless-midi-dress-5ec6e9639c3378dd28e09dc1
https://poshmark.com/listing/Rick-Owens-Classic-Biker-Moto-Suede-Jacket-Grey-5ec284c7bcbb521fee3fc24a
https://poshmark.com/listing/Rick-Owens-Archive-Vintage-04-Black-Cardigan-L-5e9eee02f01bb266b61b5859
https://poshmark.com/listing/Rick-Owens-Rare-Black-Knit-L-MSRP-1300-5e9ef1a66ab2f40bf9293819
https://poshmark.com/listing/Rick-Owens-DRKSHDW-Angora-Blend-Knit-Wrap-Top-5ecc1bef463d4fbdb25c688b
https://poshmark.com/listing/Rick-Owens-black-leather-sleeve-cone-coat-sz-10-5bd4f2c29fe48669a7cf04d8
https://poshmark.co

https://poshmark.com/listing/Rick-Owens-Wool-Silk-Lined-Jacket-US4-5ed4498547e0e4df2844f2f9
https://poshmark.com/listing/Rick-Owens-Black-Leather-Shearling-Jacket-US-6-5ed447cb8d9be9a2b4e81be1
5
https://poshmark.com/listing/Rick-Owens-DRKSHDW-duffle-bag-5e61c1e67a8173c69b1f9a3e
https://poshmark.com/listing/Rick-Owens-Lilies-lightweight-jacket-5c9009945bbb80264ec746a6
https://poshmark.com/listing/Rick-Owens-Naska-Brushed-Lamb-Leather-Jacket-Pearl-5a1e83c04de0c1605302862a
https://poshmark.com/listing/New-Rick-Owens-pullover-oversized-tunic-sweater-5e0ab60e74af4749a05697e5
https://poshmark.com/listing/Rick-Owens-Vicious-SS14-Kimono-Sleeveless-Jacket-5ebf4f4118013659120ae333
https://poshmark.com/listing/Rick-Owens-asymmetric-funnel-neck-cutaway-tailcoat-5e2e0dff79df27bffa6a7a57
https://poshmark.com/listing/Rick-Owens-Black-Jacket-5dc4709cd1aa25b3144a2821
https://poshmark.com/listing/Rick-Owens-Jacket-size-6-US-Pearl-light-grey-5dc059e2ffc2d46d3246f0bb
https://poshmark.com/listing/Rick-Owen

https://poshmark.com/listing/AUTH-Rick-Owens-Blk-Cotton-Beige-Leather-Jacket-597b9f9d1e08adbed51ade87
https://poshmark.com/listing/NWT-RICK-OWENS-PARCHMENT-LEATHER-Moto-jacket-coat-5ddaf3fdc2835c6e05f5d10d
9
https://poshmark.com/listing/Transit-longline-bomber-coat-Rick-Owens-style-S-5d990731d948a1650a57d152
https://poshmark.com/listing/New-Rick-Owens-Merino-wool-sweater-5c8fd8e1534ef9b4329d493e
https://poshmark.com/listing/Rick-Owens-Cowl-neckline-dress-5ea1f321e1d990c7db01c6e0
https://poshmark.com/listing/Rick-Owens-SS13-Island-Blouse-5ed16e1c47e0e468d2403c3d
https://poshmark.com/listing/Rick-Owens-Naska-Shearling-in-Dark-Dusk-5e3b4c2b2cc5155d86935c10
https://poshmark.com/listing/RICK-OWENS-x-ADIDAS-Runner-Sneakers-5bfc09a1df03078971dd7335
https://poshmark.com/listing/Rick-Owens-jeans-black-skinny-5e6502b726219f018faa83a4
https://poshmark.com/listing/Rick-Owens-Blistered-Lambs-Leather-Jacket-5ed1507f81a36cd7df384e19
https://poshmark.com/listing/Rick-Owens-Jacket-5d2fc2ff7a817373b84a3

https://poshmark.com/listing/Rick-OwensMetallic-gold-money-belt-pouch-OS-5d6173462f4831aadf90667d
https://poshmark.com/listing/Rick-Owens-DRKHSW-mens-sweatshirt-XS-size-5c3a272e9539f7950197b2d9
https://poshmark.com/listing/Rick-Owens-DRKSHDW-mens-jeans-size-28-5da1ceaf6a7fba58acc271b5
https://poshmark.com/listing/Rick-Owens-DRKHSW-mens-sweatshirt-no-sleeves-S-5bf99293f63eea6d18280714
https://poshmark.com/listing/Rick-Owens-DRKSHDW-mens-sweatshirt-size-S-5b9d13456a0bb70e102b9101
https://poshmark.com/listing/Rick-Owens-DRKSHDW-mens-tshirt-Size-S-5ba2712da5d7c64616c42edd
https://poshmark.com/listing/Rick-Owens-women-crossbody-bag-5d6d5c0eafade8c3ce9d4d53
https://poshmark.com/listing/Rick-Owens-women-shearling-booty-Size-41USA-11-5c2141f2fe5151bb6f458e5b
https://poshmark.com/listing/Rick-Owens-womens-boot-Size-41-USA-11-5c83e122aa877008d56ecf79
https://poshmark.com/listing/Rick-Owens-women-sneaker-Size-41-5dc8b06bbb22e34117a1f3e2
https://poshmark.com/listing/Rick-Owens-cashmere-women-turtl

https://poshmark.com/listing/RICK-OWENS-new-wrap-halter-top-army-green-small-5bbc2df83c98443ff70f1638
https://poshmark.com/listing/Rick-Owens-lilies-dinge-Draped-halter-top-new-5b9f8cf64ab633ba5af1b285
https://poshmark.com/listing/RICK-OWENS-BIKER-JACKET-5e595cf52eb33f51c5993dc8
14
https://poshmark.com/listing/Rick-Owens-top-5e9b5f01045e39b48fdd37ff
https://poshmark.com/listing/Rick-Owens-Morse-Dagger-Wool-Jacket-5c5f64660cb5aa941ed2ed5e
https://poshmark.com/listing/Rick-Owens-Black-Leather-Sandals-Sz-39-5e7d6d922a1effcd40e9cc19
https://poshmark.com/listing/Rick-Owens-bag-5ea5228888cce33ce1d36b03
https://poshmark.com/listing/Rick-Owens-zip-up-top-5cc97a1da20dfcaa58638d37
https://poshmark.com/listing/Rick-Owens-original-leather-jacket-5ea1c0818da5c9cb97acc743
https://poshmark.com/listing/Rick-Owens-tie-collar-jersey-dress-38-5e0ccc105967042ec8294121
https://poshmark.com/listing/Rick-Owens-Lillies-leather-jacket-5b66146b7386bc6f5bf7f6d7
https://poshmark.com/listing/Rick-Owens-top-size-US

...

https://poshmark.com/listing/Rick-Owens-DRKSHDW-Mens-Jeans31-5cd0f1cafe19c784f4a74f74
https://poshmark.com/listing/Rick-Owens-Gray-Geowebbed-Parka-5a1df289f0137d794213065d
https://poshmark.com/listing/Drawstring-Cargo-Pants-Rick-Owens-look-alike-5e76b5d8264a55d490de0b7c
https://poshmark.com/listing/Rick-Owens-black-halter-tank-top-shirt-minimalist-5d4bd928969d1f6a37194b77
https://poshmark.com/listing/Rick-Owens-Long-trench-coat-cotton-vintage-black-5cab973c9d3b78bb099b265e
https://poshmark.com/listing/Rick-Owens-Thigh-Leather-Wedges-375-5e77c5b6b3e9177a623cd5cc
https://poshmark.com/listing/Rick-Owens-Brand-NEW-jeans-5e548f15689ebc746cc5ecef
20
https://poshmark.com/listing/NWT-Rick-Owens-Black-Silk-Bomber-Jacket-5ce843d229f030cdeb23eda1
https://poshmark.com/listing/Rick-Owens-dna-dust-trousers-5e72d88810f00f4d27b1afa0
https://poshmark.com/listing/Rick-Owens-drop-crotch-wool-5e62fe212cc5153335dab50b
https://poshmark.com/listing/Rick-Owens-sweatpants-5e62fbcfadb58d4c5db35312
https://poshm

https://poshmark.com/listing/RICK-OWENS-LILIES-black-dress-wsuede-detail-sz-40-5e31c05375bd8ebee86a6a5b
https://poshmark.com/listing/Rick-Owens-Wool-Jacket-5e23d08fe974fb513c2497c2
https://poshmark.com/listing/RICK-OWENS-Short-dress-5cecbc5f9ed36dcce561944d
https://poshmark.com/listing/RICK-OWENS-dress-5cecc361bb22e3ccd81d06a7
https://poshmark.com/listing/RICK-OWENS-mini-dress-5d2b6b0f08d2c2b721e29a67
https://poshmark.com/listing/SALE-Rick-Owens-Drkshdw-Waxed-Cropped-Jeans-5e306e9b1052932f3860b9be
https://poshmark.com/listing/SALE-Rick-Owens-KeyChain-5e306d8fcfb5fc4357cd9770
https://poshmark.com/listing/SALE-Rick-Owens-Drkshdw-Pods-Shorts-5e306b3c01e20414c398b47a
https://poshmark.com/listing/SALE-Rick-Owens-Soultrains-Pants-5e306962582af8b3cc1f4588
https://poshmark.com/listing/Rick-Owens-textured-leather-ankle-boots-5e2d1b2622bd7cd7ed352954
https://poshmark.com/listing/Rick-Owens-Peaked-Back-Slouchy-Boots-5e2c9ebd10f00fa3247d69f5
https://poshmark.com/listing/RICK-OWENS-High-Top-Suede-L

https://poshmark.com/listing/RICK-OWENS-SKIRT-5da7a1eb9ed36db58a5973f0
https://poshmark.com/listing/RICK-OWENS-Black-Leather-Ankle-Heels-Boots-Italy-5d7d863b9ed36df01ecd35b1
https://poshmark.com/listing/Rick-Owens-Bucket-Bag-5da7cfdf7f617f6caefd9ffb
https://poshmark.com/listing/Issey-Miyake-hobo-tote-bag-Yohji-Rick-Owens-Comme-5da67b79dce03267b6e354a1
https://poshmark.com/listing/Rick-Owens-leather-jacket-5c37afc3c61777d4a57fb0d5
https://poshmark.com/listing/Rick-Owens-Inspired-Distressed-Cardigan-TOPMAN-5ce5c3222eb33f41044b2ce0
https://poshmark.com/listing/Rick-Owens-Macassar-sweater-size-M-5da4137a7f617fc84bc7c37e
https://poshmark.com/listing/RICK-OWENS-DRKSHDW-SS16-CYCLOPS-TANK-IN-DARKDUST-5d9cd9e5cfd124b3932fc5dc
https://poshmark.com/listing/Rick-Owens-light-jacket-59a5aea1bf6df5a171018223
https://poshmark.com/listing/Adidas-x-Rick-Owens-Men-Level-Runner-Low-II-5d7808d162e9d51e34d987a3
https://poshmark.com/listing/Rick-Owens-DRKSHDW-5d927ee619c15713eaba23bb
https://poshmark.com/lis

https://poshmark.com/listing/Rick-Owens-Lamb-Skin-Sleeve-Denim-Body-Jacket-5c4795c49519962c54dd7211
https://poshmark.com/listing/Rick-Owens-DRKSHDW-Replacement-Shoelaces-5c6df0503f1a0dadfcea7c5b
27
https://poshmark.com/listing/Rick-Owens-Memphis-jeans-5c6727f8d365be26eff7c250
https://poshmark.com/listing/Rick-Owens-like-new-comfortable-laceless-sneakers-5c572f8d409c15b67ad6b7e6
https://poshmark.com/listing/Rick-Owens-Black-Wool-Blazer-5c52fbc75fef37b9bb2fd59a
https://poshmark.com/listing/DRKSHDW-by-Rick-Owens-Black-Torino-Cut-Jeans-SZ28-5c4e54ff42aa763899014794
https://poshmark.com/listing/Rick-Owens-Lambskin-Leather-Jacket-5c4cc1c2b6a942b0de99647e
https://poshmark.com/listing/Rick-Owens-5bc2a9a3c617778805701daf
https://poshmark.com/listing/Rick-Owens-5c4126ce194dad1b2035ab6e
https://poshmark.com/listing/SALERick-Owens-Bonnie-Blouse-in-Dust-NWT-583f9da0713fde111a00d484
https://poshmark.com/listing/Rick-Owens-Multiway-vest-5c3e8222a5d7c650402859c2
https://poshmark.com/listing/Rick-Owens

https://poshmark.com/listing/Rick-Owens-Lamb-Leather-Vest-57ac098db4188e62e9012184
https://poshmark.com/listing/Rick-Owens-Adidas-Beige-59754cb92de512203c053a62
https://poshmark.com/listing/Rick-Owens-DRKSHDW-Long-Hoodie-Size-S-5928e048fbf6f96fd801550f
https://poshmark.com/listing/Rick-Owens-Adidas-Springblade-leather-sneaker-105-592876deeaf030e122002ba4
https://poshmark.com/listing/Mens-Rick-Owens-US-Sz-85-59234ed96a58309ddf0130e3
https://poshmark.com/listing/Rick-Owens-capris-5923c07336d59431730eb34d
https://poshmark.com/listing/Rick-Owens-black-silk-gown-size-s-580297a12ba50ae18d066a37
https://poshmark.com/listing/Rick-Owens-Adidas-Mastadon-fits-like-a-910-58cdd24c4e95a30e4f0948d8
https://poshmark.com/listing/Rick-Owens-tunic-59094e6e99086a665e01127b
https://poshmark.com/listing/Rick-Owens-59094d712de5126496011031
https://poshmark.com/listing/Rick-Owens-size-9-59002288bf6df5b9eb007860
https://poshmark.com/listing/Rick-Owens-Geobasket-58f4dbc4b4188e89c800b7d8
https://poshmark.com/lis